In [2]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from tools import web_search_tool, save_report_to_md

In [ ]:
model_client = OpenAIChatCompletionClient(
    # model="gpt-4.1-nano",  # bigger context window - rate limit
    # model="gpt-5-nano", # rate limit
    model="gpt-4o-mini",
)

In [4]:
research_planner = AssistantAgent(
    "research_planner",
    description="A strategic research coordinator that breaks down complex questions into research subtasks",
    model_client=model_client,
    system_message="""You are a research planning specialist. Your job is to create a focused research plan.

For each research question, create a FOCUSED research plan with:

1. **Core Topics**: 2-3 main areas to investigate
2. **Search Queries**: Create 3-5 specific search queries covering:
   - Latest developments and news
   - Key statistics or data
   - Expert analysis or studies
   - Future outlook

Keep the plan focused and achievable. Quality over quantity.""",
)

research_agent = AssistantAgent(
    "research_agent",
    description="A web research specialist that searches and extracts content",
    tools=[web_search_tool],
    model_client=model_client,
    system_message="""You are a web research specialist. Your job is to conduct focused searches based on the research plan.

RESEARCH STRATEGY:
1. **Execute 3-5 searches** from the research plan
2. **Extract key information** from the results:
   - Main facts and statistics
   - Recent developments
   - Expert opinions
   - Important context

3. **Quality focus**:
   - Prioritize authoritative sources
   - Look for recent information (within 2 years)
   - Note diverse perspectives

After completing the searches from the plan, summarize what you found. Your goal is to gather 5-10 quality sources.""",
)

research_analyst = AssistantAgent(
    "research_analyst",
    description="An expert analyst that creates research reports",
    model_client=model_client,
    system_message="""You are a research analyst. Create a comprehensive report from the gathered research.

CREATE A RESEARCH REPORT with:

## Executive Summary
- Key findings and conclusions
- Main insights

## Background & Current State
- Current landscape
- Recent developments
- Key statistics and data

## Analysis & Insights
- Main trends
- Different perspectives
- Expert opinions

## Future Outlook
- Emerging trends
- Predictions
- Implications

## Sources
- List all sources used

Write a clear, well-structured report based on the research gathered. End with "REPORT_COMPLETE" when finished.""",
)

quality_reviewer = AssistantAgent(
    "quality_reviewer",
    description="A quality assurance specialist that evaluates research completeness and accuracy",
    tools=[save_report_to_md],
    model_client=model_client,
    system_message="""You are a quality reviewer. Your job is to check if the research analyst has produced a complete research report.

Look for:
- A comprehensive research report from the research analyst that ends with "REPORT_COMPLETE"
- The research question is fully answered
- Sources are cited and reliable
- The report includes summary, key information, analysis, and sources

When you see a complete research report that ends with "REPORT_COMPLETE":
1. First, use the save_report_to_md tool to save the report to report.md
2. Then say: "The research is complete. The report has been saved to report.md. Please review the report and let me know if you approve it or need additional research."

If the research analyst has NOT yet created a complete report, tell them to create one now.""",
)

research_enhancer = AssistantAgent(
    "research_enhancer",
    description="A specialist that identifies critical gaps only",
    model_client=model_client,
    system_message="""You are a research enhancement specialist. Your job is to identify ONLY CRITICAL gaps.

Review the research and ONLY suggest additional searches if there are MAJOR gaps like:
- Completely missing recent developments (last 6 months)
- No statistics or data at all
- Missing a crucial perspective that was specifically asked for

If the research covers the basics reasonably well, say: "The research is sufficient to proceed with the report."

Only suggest 1-2 additional searches if absolutely necessary. We prioritize getting a good report done rather than perfect coverage.""",
)

# (human reviewr)
user_proxy = UserProxyAgent(
    "user_proxy",
    description="Human reviewer who can request additional research or approve final results",
    input_func=input,
)

In [5]:
# 중요) agent를 어떻게 선택할 것인가?
selector_prompt = """
Choose the best agent for the current task based on the conversation history:

{roles}

Current conversation:
{history}

Available agents:
- research_planner: Plan the research approach (ONLY at the start)
- research_agent: Search for and extract content from web sources (after planning)
- research_enhancer: Identify CRITICAL gaps only (use sparingly)
- research_analyst: Write the final research report
- quality_reviewer: Check if a complete report exists
- user_proxy: Ask the human for feedback

WORKFLOW:
1. If no planning done yet → select research_planner
2. If planning done but no research → select research_agent  
3. After research_agent completes initial searches → select research_enhancer ONCE
4. If enhancer says "sufficient to proceed" → select research_analyst
5. If enhancer suggests critical searches → select research_agent ONCE more then research_analyst
6. If research_analyst said "REPORT_COMPLETE" → select quality_reviewer
7. If quality_reviewer asked for user feedback → select user_proxy

IMPORTANT: After research_agent has searched 2 times maximum, proceed to research_analyst regardless.

Pick the agent that should work next based on this workflow."""

In [6]:
text_termination = TextMentionTermination("APPROVED")
max_message_termination = MaxMessageTermination(max_messages=50)
termination_condition = text_termination | max_message_termination

In [7]:
team = SelectorGroupChat(
    participants=[
        research_agent,
        research_analyst,
        research_enhancer,
        research_planner,
        quality_reviewer,
        user_proxy,
    ],
    selector_prompt=selector_prompt,
    model_client=model_client,  # it's own ai model
    # allow_repeated_speaker=True,  # a agent can speak multiple times
    termination_condition=termination_condition,
)

In [8]:
await Console(team.run_stream(task="XR 산업의 현재 개발 현황."))

---------- TextMessage (user) ----------
XR 산업의 현재 개발 현황.
---------- TextMessage (research_planner) ----------
### Research Plan: Current Developments in the XR Industry

#### 1. Core Topics:
   - **Technological Advancements**: Investigate the latest innovations in XR technologies, including hardware and software improvements.
   - **Market Trends**: Analyze the current market landscape, including major players, investment trends, and consumer adoption rates.
   - **Applications Across Industries**: Explore how XR is being utilized across different sectors, such as healthcare, education, and entertainment.

#### 2. Search Queries:
   1. **Latest Developments and News**: 
      - "latest updates in XR technology 2023"
      - "top XR innovations and product launches 2023"
   2. **Key Statistics or Data**: 
      - "XR industry market size and growth statistics 2023"
      - "consumer adoption rates of XR technologies report 2023"
   3. **Expert Analysis or Studies**: 
      - "expert a

TaskResult(messages=[TextMessage(id='af0ec4a1-22d3-43e1-bb8f-50abdf73fad3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 21, 10, 26, 29, 132089, tzinfo=datetime.timezone.utc), content='XR 산업의 현재 개발 현황.', type='TextMessage'), TextMessage(id='4610e386-2ec5-40ec-8eaf-a2dceec17cbf', source='research_planner', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=284), metadata={}, created_at=datetime.datetime(2025, 9, 21, 10, 26, 35, 556774, tzinfo=datetime.timezone.utc), content='### Research Plan: Current Developments in the XR Industry\n\n#### 1. Core Topics:\n   - **Technological Advancements**: Investigate the latest innovations in XR technologies, including hardware and software improvements.\n   - **Market Trends**: Analyze the current market landscape, including major players, investment trends, and consumer adoption rates.\n   - **Applications Across Industries**: Explore how XR is being utilized across different sectors, such as he